In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('clean-brace-368802-0ccd61453878.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#
date = 0 #当日は0、前日は1・・・
#date = 1 #当日は0、前日は1・・・
date2 = "08/19"
pmc = 28013003 # 店舗ID
SPREADSHEET_KEY = '1A7Thh_VdJeIG4e-AfF4-B79rYJb7k9N4cVtxaRQBx5k' # スプレッドシートのID
pstart = 3 # パチンコの1台目
sstart = 501 # スロットの1台目
wait = 0.02

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
#s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
#s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2000":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2000":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

time.sleep(5)

SPREADSHEET_KEY = '1LnKJY7evIkHilKi4ODBHk6lK7NjyPfpmSnpLXALt7ho' # スプレッドシートのID
gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
s20row = worksheet.cell(11,  28).value

time.sleep(5)

wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/19', '025364', '3', '47436.0', '14144.0', '80', '26']
['08/19', '025603', '4', '43846.0', '-4814.0', '35', '15']
['08/19', '025521', '5', '31538.0', '3010.0', '39', '15']
['08/19', '025364', '6', '30769.0', '3010.0', '42', '17']
['08/19', '025604', '7', '39744.0', '-6018.0', '29', '18']
['08/19', '025863', '8', '44359.0', '12038.0', '83', '21']
['08/19', '025844', '9', '36410.0', '47246.0', '81', '16']
['08/19', '025659', '10', '32051.0', '-2106.0', '32', '15']
['08/19', '025514', '11', '31026.0', '2709.0', '49', '15']
['08/19', '025564', '12', '28462.0', '-1504.0', '64', '24']
['08/19', '025613', '13', '48718.0', '13242.0', '54', '16']
['08/19', '025722', '14', '28462.0', '-1203.0', '36', '8']
['08/19', '025633', '15', '30256.0', '-7221.0', '32', '11']
['08/19', '025694', '16', '25128.0', '3913.0', '35', '13']
['08/19', '025831', '17', '37692.0', '11135.0', '64', '19']
['08/19', '025727', '18', '28462.0', '9029.0', '31', '12']
['08/19', '025865', '19', '30256.0', '3311.0', '92', 

['08/19', '025864', '154', '3333.0', '-3008.0', '0', '0']
['08/19', '025665', '155', '23846.0', '-7823.0', '10', '4']
['08/19', '025856', '156', '10256.0', '3311.0', '8', '2']
['08/19', '025879', '157', '0', '0', '0', '0']
['08/19', '025779', '158', '4615.0', '-902.0', '5', '1']
['08/19', '025841', '159', '14872.0', '-7823.0', '3', '1']
['08/19', '025782', '160', '15128.0', '24978.0', '57', '5']
['08/19', '025897', '161', '21282.0', '-5115.0', '14', '4']
['08/19', '025879', '162', '1282.0', '-1203.0', '0', '0']
['08/19', '025652', '163', '5641.0', '-3911.0', '1', '1']
['08/19', '025667', '164', '1795.0', '-1504.0', '0', '0']
['08/19', '025876', '165', '1026.0', '-1203.0', '1', '1']
['08/19', '025865', '166', '10256.0', '-4513.0', '12', '4']
['08/19', '025417', '167', '13077.0', '1806.0', '19', '6']
['08/19', '025564', '168', '12821.0', '7223.0', '67', '6']
['08/19', '025885', '169', '3333.0', '-2406.0', '4', '1']
['08/19', '025639', '170', '7436.0', '11436.0', '26', '3']
['08/19', '025

['08/19', '025765', '305', '38462.0', '-1504.0', '41', '14']
['08/19', '025765', '306', '48974.0', '-4212.0', '49', '23']
['08/19', '120137', '501', '4419', '1065.0', '17', '7', '24']
['08/19', '120033', '502', '2680', '-694.0', '8', '3', '--']
['08/19', '120149', '503', '1599', '-787.0', '0', '3', '14']
['08/19', '120151', '504', '3267', '1667.0', '0', '4', '18']
['08/19', '120146', '505', '1812', '-880.0', '0', '4', '3']
['08/19', '119945', '506', '1994', '-139.0', '6', '3', '--']
['08/19', '120120', '507', '76', '0', '0', '0', '--']
['08/19', '120108', '508', '4731', '2870.0', '0', '0', '49']
['08/19', '119984', '509', '1468', '-648.0', '3', '2', '--']
['08/19', '120131', '510', '3164', '-2824.0', '0', '1', '34']
['08/19', '120076', '511', '1440', '-926.0', '0', '0', '6']
['08/19', '119972', '512', '2955', '694.0', '8', '5', '--']
['08/19', '120033', '513', '5426', '3287.0', '24', '17', '--']
['08/19', '119940', '514', '3051', '-139.0', '15', '0', '12']
['08/19', '120077', '515', '3

['08/19', '120010', '634', '7010', '1343.0', '30', '27', '--']
['08/19', '120010', '635', '7406', '1111.0', '30', '25', '--']
['08/19', '120010', '636', '4190', '-926.0', '12', '10', '--']
['08/19', '120010', '637', '5491', '-2222.0', '12', '14', '--']
['08/19', '120010', '638', '7422', '324.0', '30', '20', '--']
['08/19', '120010', '639', '4282', '-1111.0', '14', '6', '--']
['08/19', '120010', '640', '6364', '1759.0', '28', '22', '--']
['08/19', '120010', '641', '7591', '3519.0', '40', '27', '--']
['08/19', '120010', '642', '4732', '-1296.0', '11', '16', '--']
['08/19', '120010', '643', '6196', '-139.0', '24', '18', '--']
['08/19', '120010', '644', '6077', '-1574.0', '19', '11', '--']
['08/19', '120010', '645', '6419', '1481.0', '31', '14', '--']
['08/19', '120010', '646', '8673', '1852.0', '38', '26', '--']
['08/19', '120010', '647', '6142', '-880.0', '18', '20', '--']
['08/19', '120010', '648', '2781', '-278.0', '13', '1', '--']
['08/19', '120010', '649', '7585', '1759.0', '39', '17

['08/19', '120073', '766', '2175', '-2407.0', '0', '4', '5']
['08/19', '120073', '767', '2875', '370.0', '0', '8', '19']
['08/19', '120073', '768', '6012', '-3241.0', '0', '18', '25']
['08/19', '120073', '769', '5059', '-2037.0', '0', '17', '26']
['08/19', '120111', '770', '2946', '-324.0', '15', '10', '--']
['08/19', '120111', '771', '4168', '-1991.0', '17', '11', '--']
['08/19', '120111', '772', '2538', '-787.0', '11', '9', '--']
['08/19', '120111', '773', '4102', '278.0', '27', '10', '--']
['08/19', '120111', '774', '5273', '-3843.0', '18', '7', '--']
['08/19', '120111', '775', '3551', '93.0', '24', '8', '--']
['08/19', '120111', '776', '3400', '-556.0', '18', '10', '--']
['08/19', '120111', '777', '5435', '-3287.0', '22', '9', '--']
['08/19', '120111', '778', '1670', '185.0', '12', '2', '--']
['08/19', '120111', '779', '3911', '-1389.0', '17', '12', '--']
['08/19', '120111', '780', '2589', '2176.0', '27', '5', '--']
['08/19', '120111', '781', '4823', '-1435.0', '22', '13', '--']
['

{'spreadsheetId': '1LnKJY7evIkHilKi4ODBHk6lK7NjyPfpmSnpLXALt7ho',
 'updates': {'spreadsheetId': '1LnKJY7evIkHilKi4ODBHk6lK7NjyPfpmSnpLXALt7ho',
  'updatedRange': 'S20RAW!F10200:M10537',
  'updatedRows': 338,
  'updatedColumns': 8,
  'updatedCells': 2704}}

In [8]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
#s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2000":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2000":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

time.sleep(5)

SPREADSHEET_KEY = '1LnKJY7evIkHilKi4ODBHk6lK7NjyPfpmSnpLXALt7ho' # スプレッドシートのID
gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
s20row = worksheet.cell(11,  28).value

time.sleep(5)

wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['07/21', '025364', '3', '37692.0', '-4814.0', '39', '14']
['07/21', '025603', '4', '13846.0', '-2406.0', '11', '5']
['07/21', '025521', '5', '27179.0', '-902.0', '32', '11']
['07/21', '025364', '6', '27949.0', '-6619.0', '24', '13']
['07/21', '025605', '7', '11795.0', '603.0', '13', '3']
['07/21', '025844', '8', '29487.0', '16552.0', '44', '8']
['07/21', '025659', '10', '36410.0', '-6319.0', '36', '13']
['07/21', '025514', '11', '28205.0', '-2406.0', '32', '7']
['07/21', '025564', '12', '27949.0', '3612.0', '79', '18']
['07/21', '025613', '13', '37179.0', '9330.0', '36', '12']
['07/21', '025722', '14', '45128.0', '-4513.0', '50', '21']
['07/21', '025633', '15', '20513.0', '-3911.0', '24', '9']
['07/21', '025694', '16', '14103.0', '-2106.0', '13', '8']
['07/21', '025831', '17', '36923.0', '2408.0', '42', '16']
['07/21', '025727', '18', '20769.0', '302.0', '22', '8']
['07/21', '025369', '19', '45641.0', '4515.0', '99', '40']
['07/21', '025225', '20', '41795.0', '2107.0', '48', '42']
['0

['07/21', '025779', '158', '5385.0', '-4513.0', '0', '0']
['07/21', '025841', '159', '10513.0', '302.0', '9', '3']
['07/21', '025782', '160', '7179.0', '10232.0', '21', '3']
['07/21', '025847', '161', '0', '0', '0', '0']
['07/21', '025879', '162', '6154.0', '8728.0', '15', '3']
['07/21', '025652', '163', '0', '0', '0', '0']
['07/21', '025667', '164', '2308.0', '-1805.0', '0', '0']
['07/21', '025876', '165', '28718.0', '-9328.0', '20', '7']
['07/21', '025847', '166', '2308.0', '-902.0', '6', '1']
['07/21', '025417', '167', '7179.0', '-1504.0', '6', '4']
['07/21', '025564', '168', '1026.0', '-1203.0', '1', '1']
['07/21', '025885', '169', '15641.0', '6922.0', '68', '3']
['07/21', '025639', '170', '13590.0', '1506.0', '20', '5']
['07/21', '025590', '171', '0', '0', '0', '0']
['07/21', '025590', '172', '5897.0', '-902.0', '6', '6']
['07/21', '025590', '173', '19487.0', '2408.0', '27', '27']
['07/21', '025590', '174', '1795.0', '302.0', '2', '2']
['07/21', '025603', '179', '22308.0', '2408.0

['07/21', '120149', '503', '6243', '-648.0', '0', '21', '74']
['07/21', '120132', '504', '519', '-556.0', '1', '0', '--']
['07/21', '120146', '505', '4107', '3889.0', '0', '8', '17']
['07/21', '119945', '506', '2477', '-463.0', '5', '4', '--']
['07/21', '120120', '507', '2719', '-2037.0', '0', '0', '11']
['07/21', '120108', '508', '511', '-185.0', '0', '0', '2']
['07/21', '119984', '509', '2333', '417.0', '9', '8', '--']
['07/21', '120131', '510', '2580', '556.0', '0', '14', '69']
['07/21', '120076', '511', '747', '-417.0', '0', '0', '2']
['07/21', '119972', '512', '942', '-787.0', '1', '2', '--']
['07/21', '120033', '513', '4714', '-1065.0', '15', '9', '--']
['07/21', '119940', '514', '1821', '-880.0', '6', '0', '5']
['07/21', '120077', '515', '3644', '648.0', '0', '21', '38']
['07/21', '120036', '516', '3553', '-972.0', '9', '8', '--']
['07/21', '120139', '517', '703', '-93.0', '0', '1', '4']
['07/21', '120034', '518', '799', '880.0', '0', '2', '11']
['07/21', '120146', '519', '3239'

['07/21', '120096', '658', '4072', '-648.0', '10', '16', '--']
['07/21', '120096', '659', '1612', '93.0', '8', '2', '--']
['07/21', '120096', '660', '2905', '-972.0', '8', '6', '--']
['07/21', '120096', '661', '1588', '-787.0', '4', '1', '--']
['07/21', '120096', '662', '2702', '-93.0', '13', '4', '--']
['07/21', '120096', '663', '2969', '-324.0', '10', '8', '--']
['07/21', '120096', '664', '1485', '-833.0', '2', '4', '--']
['07/21', '120096', '665', '2439', '-972.0', '7', '6', '--']
['07/21', '120096', '666', '2822', '-417.0', '10', '6', '--']
['07/21', '120096', '667', '1328', '-1157.0', '2', '1', '--']
['07/21', '120010', '668', '4859', '694.0', '23', '10', '--']
['07/21', '120010', '669', '7203', '139.0', '28', '18', '--']
['07/21', '120010', '670', '4388', '-1019.0', '14', '9', '--']
['07/21', '120010', '671', '4024', '-463.0', '12', '11', '--']
['07/21', '120010', '672', '3741', '185.0', '15', '10', '--']
['07/21', '120010', '673', '7534', '-93.0', '25', '28', '--']
['07/21', '12

['07/21', '120073', '792', '3775', '-1157.0', '0', '11', '20']
['07/21', '120073', '793', '2943', '-648.0', '0', '5', '18']
['07/21', '120073', '794', '3358', '-1389.0', '0', '9', '17']
['07/21', '120073', '795', '3584', '-1806.0', '0', '6', '19']
['07/21', '120073', '796', '3085', '2731.0', '0', '7', '35']
['07/21', '120073', '797', '3379', '-463.0', '0', '8', '24']
['07/21', '120073', '798', '5611', '-139.0', '0', '18', '40']
['07/21', '120127', '799', '2328', '-2037.0', '0', '0', '6']
['07/21', '120127', '800', '32', '0', '0', '0', '--']
['07/21', '120127', '801', '2188', '139.0', '0', '0', '4']
['07/21', '120127', '802', '30', '0', '0', '0', '--']
['07/21', '120127', '803', '984', '-1111.0', '0', '0', '1']
['07/21', '120127', '804', '37', '0', '0', '0', '--']
['07/21', '120127', '805', '28', '0', '0', '0', '--']
['07/21', '120127', '806', '854', '-509.0', '0', '0', '1']
['07/21', '119926', '807', '4321', '-2963.0', '0', '1', '23']
['07/21', '119966', '808', '788', '-602.0', '0', '2

{'spreadsheetId': '1LnKJY7evIkHilKi4ODBHk6lK7NjyPfpmSnpLXALt7ho',
 'updates': {'spreadsheetId': '1LnKJY7evIkHilKi4ODBHk6lK7NjyPfpmSnpLXALt7ho',
  'updatedRange': 'S20RAW!F962:M1281',
  'updatedRows': 320,
  'updatedColumns': 8,
  'updatedCells': 2560}}